In [40]:
import pandas as pd
import geopandas as gpd


# Load geospatial data for Berlin postal codes
df_geodat_plz = pd.read_csv("../data/geodata_berlin_plz.csv", delimiter=";")

# Load electric charging station data from an Excel file
df_lstat = pd.read_excel("../data/Ladesaeulenregister_SEP.xlsx")


def sort_by_plz_add_geometry(dfr, dfg): 
    dframe                  = dfr.copy()
    df_geo                  = dfg.copy()
    
    sorted_df               = dframe\
        .sort_values(by='PLZ')\
        .reset_index(drop=True)\
        .sort_index()
        
    sorted_df2              = sorted_df.merge(df_geo, on='PLZ', how ='left')
    sorted_df3              = sorted_df2.dropna(subset=['geometry'])
    
    sorted_df3['geometry']  = gpd.GeoSeries.from_wkt(sorted_df3['geometry'])
    ret                     = gpd.GeoDataFrame(sorted_df3, geometry='geometry')
    
    return ret



def preprocess_lstat(dfr, dfg):
    """Preprocessing dataframe from Ladesaeulenregister_SEP.xlsx"""
    dframe                  = dfr.copy()
    df_geo                  = dfg.copy()
    
    dframe2               	= dframe.loc[:,['Betreiber','Anzeigename (Karte)','Postleitzahl', 'Bundesland', 'Breitengrad', 'Längengrad', 'Nennleistung Ladeeinrichtung [kW]']]
    dframe2.rename(columns  = {'Betreiber':'stationOperator','Anzeigename (Karte)':'stationName',"Nennleistung Ladeeinrichtung [kW]":"KW", "Postleitzahl": "PLZ",  'Breitengrad':'Latitude', 'Längengrad':'Longitude', }, inplace = True)

    # Convert to string
    dframe2['Latitude']  = dframe2['Latitude'].astype(str)
    dframe2['Longitude']   = dframe2['Longitude'].astype(str)
    dframe2['stationOperator']   = dframe2['stationOperator'].astype(str)
    dframe2['stationName']   = dframe2['stationName'].astype(str)


    # Now replace the commas with periods
    dframe2['Latitude']  = dframe2['Latitude'].str.replace(',', '.')
    dframe2['Longitude']   = dframe2['Longitude'].str.replace(',', '.')

    dframe3                 = dframe2[(dframe2["Bundesland"] == 'Berlin') & 
                                            (dframe2["PLZ"] > 10115) &  
                                            (dframe2["PLZ"] < 14200)]
    
    ret = sort_by_plz_add_geometry(dframe3, df_geo)

    # Add an ID column with row numbers starting from 1
    ret['stationID'] = range(1, len(ret) + 1)

    # Add a blank column named 'Remarks' (all values set to empty strings)
    ret['ratingID'] = ''
    
    return ret


df = preprocess_lstat(df_lstat, df_geodat_plz)

df.to_csv("../data/ChargingStationData.csv.csv", index=False)

C:\Users\aka87578\AppData\Local\Temp\ipykernel_7008\2803414139.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_df3['geometry']  = gpd.GeoSeries.from_wkt(sorted_df3['geometry'])


In [41]:
df

,stationOperator,stationName,PLZ,Bundesland,Latitude,Longitude,KW,geometry,stationID,ratingID
0,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,10117,Berlin,52.51115,13.38952,30.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",1,
1,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,10117,Berlin,52.513083,13.389694,22.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",2,
2,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.515659,13.389829,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",3,
3,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.51897,13.390248,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",4,
4,E.ON Drive Infrastructure GmbH,E.ON Drive Infrastructure,10117,Berlin,52.520389,13.393959,44.0,"POLYGON ((13.40031 52.5233, 13.40051 52.52216,...",5,
...,...,...,...,...,...,...,...,...,...,...
2883,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.479241,13.308733,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2874,
2884,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.476902,13.287843,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2875,
2885,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.479021,13.292588,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2876,
2886,Berliner Stadtwerke KommunalPartner GmbH,Berliner Stadtwerke KommunalPartner GmbH,14199,Berlin,52.484282,13.294078,22.0,"POLYGON ((13.31186 52.48048, 13.30158 52.47452...",2877,


## firebase

In [47]:
import firebase_admin
from firebase_admin import credentials, auth, db
import hashlib
from datetime import datetime

In [66]:
# Initialize Firebase
cred = credentials.Certificate("../secret/firebase.json")  # Replace with your Firebase JSON file
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://ase-charging-default-rtdb.europe-west1.firebasedatabase.app/'  # Replace with your Firebase Database URL
})





ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [53]:


# Hash passwords for security
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def user_signup():
    username = input("Enter a username: ").strip()
    password = input("Enter a password: ").strip()
    
    # Hash the password
    hashed_password = hash_password(password)
    
    try:
        # Check if username already exists
        users_ref = db.reference('users')
        users = users_ref.get() or {}
        for user_id, user_data in users.items():
            if user_data['username'] == username:
                print("Username already exists. Please choose another.")
                return
        
        # Create a new user
        user_id = f"user_{len(users) + 1}"
        users_ref.child(user_id).set({
            "username": username,
            "password": hashed_password,
            "date_joined": datetime.now().isoformat()
        })
        print(f"User {username} signed up successfully!")
    except Exception as e:
        print(f"Error signing up user: {e}")



In [54]:
user_signup()

Username already exists. Please choose another.


In [58]:
def user_login():
    username = input("Enter your username: ").strip()
    password = input("Enter your password: ").strip()
    
    # Hash the input password
    hashed_password = hash_password(password)
    
    try:
        users_ref = db.reference('users')
        users = users_ref.get() or {}
        for user_id, user_data in users.items():
            if user_data['username'] == username and user_data['password'] == hashed_password:
                print(f"Welcome back, {username}!")
                return user_id  # Return the user ID for session tracking
        print("Invalid username or password.")
        return None
    except Exception as e:
        print(f"Error logging in: {e}")
        return None


In [59]:
user_login()

Welcome back, shreya!


'user_2'

In [60]:
def add_rating(user_id):
    if not user_id:
        print("You need to log in to give a rating.")
        return
    
    station_id = input("Enter the charging station ID: ").strip()
    review_star = int(input("Enter your rating (1-5 stars): ").strip())
    review_text = input("Enter your review: ").strip()
    
    if review_star < 1 or review_star > 5:
        print("Invalid rating. Please provide a rating between 1 and 5 stars.")
        return
    
    try:
        ratings_ref = db.reference('ratings')
        ratings_ref.push({
            "user_id": user_id,
            "charging_station_id": station_id,
            "review_star": review_star,
            "review_text": review_text,
            "review_date": datetime.now().isoformat()
        })
        print("Thank you for your review!")
    except Exception as e:
        print(f"Error adding review: {e}")


In [63]:
if __name__ == "__main__":
    current_user_id = None
    print("Welcome to the Charging Station Review System")
    while True:
        print("\nOptions:")
        print("1. Sign up")
        print("2. Log in")
        print("3. Add a review")
        print("4. Exit")
        choice = input("Enter your choice: ").strip()
        
        if choice == "1":
            user_signup()
        elif choice == "2":
            current_user_id = user_login()
        elif choice == "3":
            add_rating(current_user_id)
        elif choice == "4":
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")


Welcome to the Charging Station Review System

Options:
1. Sign up
2. Log in
3. Add a review
4. Exit
You need to log in to give a rating.

Options:
1. Sign up
2. Log in
3. Add a review
4. Exit
Welcome back, shreya!

Options:
1. Sign up
2. Log in
3. Add a review
4. Exit
Thank you for your review!

Options:
1. Sign up
2. Log in
3. Add a review
4. Exit
Goodbye!


In [64]:
def fetch_reviews(charging_station_id):
    try:
        # Reference the ratings in the database
        ratings_ref = db.reference('ratings')
        all_reviews = ratings_ref.get() or {}
        
        # Filter reviews for the given charging station ID
        station_reviews = [
            review for review in all_reviews.values()
            if review['charging_station_id'] == charging_station_id
        ]
        
        if not station_reviews:
            print(f"No reviews found for charging station ID {charging_station_id}.")
            return
        
        # Calculate the average star rating
        total_stars = sum(review['review_star'] for review in station_reviews)
        average_rating = total_stars / len(station_reviews)
        
        # Print the results
        print(f"Average Star Rating for Charging Station {charging_station_id}: {average_rating:.2f}")
        print("\nReviews:")
        for review in station_reviews:
            print(f"- {review['review_text']}")
    except Exception as e:
        print(f"Error fetching reviews: {e}")

# Example usage
if __name__ == "__main__":
    station_id = input("Enter the charging station ID: ").strip()
    fetch_reviews(station_id)


Average Star Rating for Charging Station 1: 4.50

Reviews:
- good
- very good
